## Demo for Xray Detection

### 1. Введение

Для решения задачи, связанной с обнаружением координат аномалий и их классификацией, я применил несколько различных подходов и технологий:

1. Библиотека MMDetection с использованием двух моделей:
   - Базовая модель RTMDet
   - Более сложная модель на основе Cascade RFP R50

2. Библиотека Ultralytics с использованием моделей YOLOv8:
   - YOLOv8m (medium)
   - YOLOv8x (extra large)

3. Библиотека Ultralytics с использованием модели YOLOv5:
   - YOLOv5x (extra large)

Каждый из этих подходов был выбран для оптимизации процесса обнаружения и классификации аномалий, учитывая их различные сильные стороны и особенности.

### 2. Предобработка данных

Учитывая ограниченный объем оригинального датасета, было принято решение оптимизировать использование имеющихся данных. Процесс подготовки данных включал следующие этапы:

1. Разделение датасета:
   - Равномерное разделение данных на тренировочную и тестовую выборки с учетом баланса классов.
   - Тестовая выборка: около 250 экземпляров.
   - Исходная тренировочная выборка: около 650 экземпляров.

2. Аугментация данных:
   - Применение методов аугментации для искусственного увеличения объема тренировочной выборки.
   - Увеличение тренировочного набора примерно в 5 раз.
   - Итоговый объем тренировочной выборки: 650 исходных + 3500 аугментированных экземпляров.

3. Результат:
   - Тестовая выборка: ~250 экземпляров.
   - Расширенная тренировочная выборка: ~4150 экземпляров (650 оригинальных + 3500 аугментированных).

Код для реализации аугментации будет представлен в следующих разделах.

In [ ]:
import os
import albumentations as A
from PIL import Image
import numpy as np

# Define the augmentation pipeline
augmentation_pipeline = A.Compose([
    A.VerticalFlip(p=0.2),
    A.HorizontalFlip(p=0.2),
    A.RandomRotate90(p=0.2),
    A.RandomBrightnessContrast(p=0.2),
    A.HueSaturationValue(p=0.2),
    A.Blur(p=0.2),
    A.GaussNoise(p=0.2),
    A.Resize(1024, 1024)
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))


def load_image_and_labels(image_path, label_path):
    image = np.array(Image.open(image_path).convert("RGB"))

    # Load labels
    bboxes = []
    class_labels = []
    with open(label_path, 'r') as file:
        for line in file.readlines():
            class_id, x_center, y_center, width, height = map(float, line.strip().split())
            bboxes.append([x_center, y_center, width, height])
            class_labels.append(int(class_id))

    return image, bboxes, class_labels


def save_augmented_data(augmented_image, augmented_bboxes, augmented_labels, output_image_path, output_label_path):
    augmented_image_pil = Image.fromarray(augmented_image)
    augmented_image_pil.save(output_image_path)

    with open(output_label_path, 'w') as file:
        for bbox, label in zip(augmented_bboxes, augmented_labels):
            bbox_str = ' '.join(map(str, bbox))
            file.write(f"{label} {bbox_str}\n")


def augment_and_save(image_path, label_path, output_image_dir, output_label_dir, num_augmentations=5):
    image, bboxes, class_labels = load_image_and_labels(image_path, label_path)
    base_name = os.path.basename(image_path)
    name, ext = os.path.splitext(base_name)

    for i in range(num_augmentations):
        augmented = augmentation_pipeline(image=image, bboxes=bboxes, class_labels=class_labels)
        augmented_image = augmented['image']
        augmented_bboxes = augmented['bboxes']
        augmented_labels = augmented['class_labels']

        output_image_path = os.path.join(output_image_dir, f"{name}_aug_{i}{ext}")
        output_label_path = os.path.join(output_label_dir, f"{name}_aug_{i}.txt")

        save_augmented_data(augmented_image, augmented_bboxes, augmented_labels, output_image_path, output_label_path)


train_images_dir = "/data/yolo_dataset/train/images"
train_labels_dir = "/data/yolo_dataset/train/labels"
augmented_images_dir = "/data/yolo_dataset/augmented/images"
augmented_labels_dir = "/data/yolo_dataset/augmented/labels"
os.makedirs(augmented_images_dir, exist_ok=True)
os.makedirs(augmented_labels_dir, exist_ok=True)

index = 0
for image_file in os.listdir(train_images_dir):
    if image_file.endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(train_images_dir, image_file)
        label_path = os.path.join(train_labels_dir,
                                  image_file.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt'))
        augment_and_save(image_path, label_path, augmented_images_dir, augmented_labels_dir, num_augmentations=5)
        print(image_path, index)
        index+=1

### 3. Проверка гипотез и тренировка модели на основе технологии mmdetection

Базовая модель на основе RTMDet была использована чтобы выроботать процесс тренировки на основе технологии mmdetection. Затем после проверки гипотез и правильного выстроенного пайплайна я решил попробовать использовать каскадную модель на основе Cascade RFP R50. Базовая конфигурация модели была использована, как показано ниже:


Эта структура будет использована для multi class detection и single class detection. Также для проверки обучаемости я добавлял и убавлял ту часть данных, которая была сгенерирована.





In [2]:
model = dict(
    roi_head=dict(
        bbox_head=[
            dict(
                type='Shared2FCBBoxHead',
                in_channels=256,
                fc_out_channels=1024,
                roi_feat_size=7,
                num_classes=8,
                bbox_coder=dict(
                    type='DeltaXYWHBBoxCoder',
                    target_means=[0., 0., 0., 0.],
                    target_stds=[0.1, 0.1, 0.2, 0.2]),
                reg_class_agnostic=True,
                loss_cls=dict(
                    type='CrossEntropyLoss',
                    use_sigmoid=False,
                    loss_weight=1.0),
                loss_bbox=dict(type='SmoothL1Loss', beta=1.0,
                               loss_weight=1.0)),
            dict(
                type='Shared2FCBBoxHead',
                in_channels=256,
                fc_out_channels=1024,
                roi_feat_size=7,
                num_classes=8,
                bbox_coder=dict(
                    type='DeltaXYWHBBoxCoder',
                    target_means=[0., 0., 0., 0.],
                    target_stds=[0.05, 0.05, 0.1, 0.1]),
                reg_class_agnostic=True,
                loss_cls=dict(
                    type='CrossEntropyLoss',
                    use_sigmoid=False,
                    loss_weight=1.0),
                loss_bbox=dict(type='SmoothL1Loss', beta=1.0,
                               loss_weight=1.0)),
            dict(
                type='Shared2FCBBoxHead',
                in_channels=256,
                fc_out_channels=1024,
                roi_feat_size=7,
                num_classes=8,
                bbox_coder=dict(
                    type='DeltaXYWHBBoxCoder',
                    target_means=[0., 0., 0., 0.],
                    target_stds=[0.033, 0.033, 0.067, 0.067]),
                reg_class_agnostic=True,
                loss_cls=dict(
                    type='CrossEntropyLoss',
                    use_sigmoid=False,
                    loss_weight=1.0),
                loss_bbox=dict(type='SmoothL1Loss', beta=1.0, loss_weight=1.0))
        ]),
    test_cfg=dict(
        rpn=dict(
            nms_across_levels=False,
            nms_pre=1000,
            nms_post=1000,
            max_num=1000,
            nms_thr=0.7,
            min_bbox_size=0),
        rcnn=dict(
            score_thr=0.001,
            nms=dict(type='nms', iou_threshold=0.5),
            max_per_img=100,
            mask_thr_binary=0.5)))

Результаты при использовании Multiple Detection на основе каскадной модели:
```
Average Precision (AP):
- AP@[IoU=0.50:0.95 | area=all | maxDets=100] = 0.061
- AP@[IoU=0.50 | area=all | maxDets=1000] = 0.117
- AP@[IoU=0.75 | area=all | maxDets=1000] = 0.060
- AP@[IoU=0.50:0.95 | area=small | maxDets=1000] = 0.000
- AP@[IoU=0.50:0.95 | area=medium | maxDets=1000] = 0.012
- AP@[IoU=0.50:0.95 | area=large | maxDets=1000] = 0.081

Average Recall (AR):
- AR@[IoU=0.50:0.95 | area=all | maxDets=100] = 0.243
- AR@[IoU=0.50:0.95 | area=all | maxDets=300] = 0.243
- AR@[IoU=0.50:0.95 | area=all | maxDets=1000] = 0.243
- AR@[IoU=0.50:0.95 | area=small | maxDets=1000] = 0.000
- AR@[IoU=0.50:0.95 | area=medium | maxDets=1000] = 0.081
- AR@[IoU=0.50:0.95 | area=large | maxDets=1000] = 0.262

COCO mAP:
bbox_mAP: 0.0610, bbox_mAP_50: 0.1170, bbox_mAP_75: 0.0600
bbox_mAP_s: 0.0000, bbox_mAP_m: 0.0120, bbox_mAP_l: 0.0810
```

### 4. Yolov8 тренировка и проверка гипотез

Для использования библиотеки ultralytics с утилизацией модели yolov8 был сделан основной файл запуска

### конфигурация для данных выглядела таким образом
```
path: "/data/new_dataset"
train:
  - "/data/new_dataset/train/images"
val: "/data/new_dataset/val/images"

names:
  0: 'Atelectasis'
  1: 'Cardiomegaly'
  2: 'Effusion'
  3: 'Infiltrate'
  4: 'Mass'
  5: 'Nodule'
  6: 'Pneumonia'
  7: 'Pneumothorax'
```


In [2]:
from ultralytics import YOLO
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

model = YOLO('yolov8x.pt')  # Use the YOLOv8 model variant you prefer

data_config = './data.yaml'
epochs = 1000
batch_size = 24
image_size = 1024
num_workers = 64
augmentation_params = {
    'flipud': 0.1,    # Vertical flip probability
    'fliplr': 0.1,    # Horizontal flip probability
    'mosaic': 0.0,    # Mosaic augmentation probability
    'mixup': 0.0,     # MixUp augmentation probability
    'hsv_h': 0.015,   # HSV-Hue augmentation
    'hsv_s': 0.1,     # HSV-Saturation augmentation
    'hsv_v': 0.1,     # HSV-Value augmentation
    'degrees': 0.1,   # Image rotation degrees
    'translate': 0.1, # Image translation
    'scale': 0.1,     # Image scaling
    'shear': 0.0,     # Image shear
    'perspective': 0.0 # Image perspective
}

model.train(data=data_config, epochs=epochs,workers=num_workers,batch=batch_size, imgsz=image_size,
            save_period=10,augment=True, device="0,1",optimizer="AdamW",project="/data/new_dataset/output_dir",name="final_xray",verbose=True,plots=True, save=True, **augmentation_params)


После тренировки общие результаты как показаны ниже:

1. mAP50-95: 0.01576 (23-я эпоха)
2. Точность: 0.76854 (32-я эпоха)
3. Полнота: 0.25541 (24-я эпоха)

Проблемы: 
1. Нестабильность в обучении
2. Функция потерь по классификации стала увеличиваться
3. Переставаемая обучаемость после определенной эпохи

### 5. Результаты

1. Было протестирована также самая последняя модель yolov5, которая так же показала хуже результаты чем yolov8. Ее показатели по mAP были ниже.
2. Искусственное увелечение датасета привело к ухудшению результатов, нежели улучшению. По этой причине в последних тренировках были использованы данные только с оригинального датасета
3. Изменение от multiple class detection на single class detection - приводило к тому, что результаты модели наоборот ухудшались, и модель стала путаться еще быстрее.
    - Общий mAP@0.5 (средняя точность при пороге IoU 0.5) составляет 0.092, что указывает на то, что производительность модели очень низкая для single класса
    - Для модели по mmdetection результаты схожи, показатели mAP при 0.5 IoU - 0.1


### 6. Что можно улучшить?
1. Увеличить датасет используя остаток данных( их там около 110К ), используя авто аннотацию на основе модели, которая была протренирована на датасете VinBigData Xray. В ней присутствуют все классы указанные в этом датасете.
2. Объединить такие классы как Mass/Nodule в один
3. Добавить weight_decay в sampler
4. Попробовать использовать FocalNet, так как она лучше работает при несбалансированных датасетах на основе FocalLoss
5. Также можно попробовать использовать DETR модель, которая на данный момент является SOTA на COCO dataset